# 1. SETTING
- 환경 설정 및 데이터 URL 불러오기

In [ ]:
# 구글 드라이브 연결
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
key = '' # 개인정보이므로 삭제했습니다.
myURI = 'http://openapi.seoul.go.kr:8088/' + key + '/xml/TbVwNature/1/1000/'

In [ ]:
print(myURI)

# 2. GET OPENAPI

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
from urllib.request import urlopen

def get_openapi(url):
  global df # 전역 변수 설정

  html = urlopen(url)
  bs = BeautifulSoup(html.read(), 'html.parser') # html.parser보다 더 깨끗한 결과 제공, 한글 스트링 호환

  itemlist = bs.find_all("row")

  df = pd.DataFrame() # 데이터를 저장할 빈 dataframe 생성

  for t in itemlist:
    try:
      language = t.find('lang_code_id').get_text()
      name = t.find('post_sj').get_text()
      address = t.find('address').get_text()
      new_address = t.find('new_address').get_text()
      tel = t.find('cmmn_telno').get_text()
      web_url = t.find('cmmn_hmpg_url').get_text()
      use_time = t.find('cmmn_use_time').get_text()
      op_info = t.find('cmmn_bsnde').get_text()
      rest_info = t.find('cmmn_rstde').get_text()
      traffic_info = t.find('subway_info').get_text()
    
    except: # 제공되지 않는(비어있는) 항목일 경우, 오류없이 ""로 받아오기 
      language = ""
      name = ""
      address = ""
      new_address = ""
      tel = ""
      web_url = ""
      use_time = ""
      op_info = ""
      rest_info = ""
      traffic_info = ""
    
    temp = pd.DataFrame(([[language, name, address, new_address, tel, web_url, use_time, op_info, rest_info, traffic_info]]), 
                        columns = ["언어", "상호명", "주소", "신주소", "전화번호", "웹사이트", 
                                    "운영시간", "운영요일", "휴무일", "교통정보"])
    df = pd.concat([df,temp])

  df = df.reset_index(drop=True)
  return df

In [ ]:
get_openapi(myURI)

,언어,상호명,주소,신주소,전화번호,웹사이트,운영시간,운영요일,휴무일,교통정보
0,ko,북한산국립공원,136-855 서울 성북구 정릉동 산 1-1,02700 서울 성북구 보국문로34길 6-21 (정릉동),02-909-0497,http://bukhan.knps.or.kr,04:00 ~ 17:00 (3월~11월)\r\n04:00 ~ 16:00 (12월~2월),매일,,"3호선 구파발역 1번 출구로 나와 704번, 34번 버스로 갈아타고 북한산성에서 하..."
1,ja,ソニュド(仙遊島)公園,150-040 ソウル市永登浦区堂山洞1,07200 ソウル市ヨンドゥンポ(永登浦)区ソンユロ343 (堂山洞),+82-2-2634-7250,http://japanese.seoul.go.kr/%E7%94%9F%E6%B4%BB...,※公園 6:00-24:00／展示館 [夏期] 9:00-18:00、[冬期] 9:00-1...,,,地下鉄6号線「ハプジョン」駅8番出口からそれぞれ5714番バスに乗り、ソニュド(仙遊島)公園...
2,ja,ソクチョン(石村)湖公園,138-915 ソウル市松坡区蚕室洞 47,05608 ソウル市ソンパグ(松坡区)サムハッサロ136 (蚕室洞),+82-2-412-0190,,,毎日,年中無休,2号線チャムシル駅2・3番出口
3,ko,중랑천 제방,서울 중랑구 면목동 일대,서울특별시 중랑구 면목동 일대,02-938-9520~1,http://jr1000ecocenter.nowon.kr/main.do,,,,"7호선 중화 3번, 4번 출구 도보 10분\r\n\r\n1호선 신이문 3번 출구 도..."
4,ko,구암공원,157-807 서울 강서구 가양동 1471,"07525 서울 강서구 허준로5길 42 (가양동, 구암공원관리사무실)",02-2600-4196,,,,,9호선 가양 1번
...,...,...,...,...,...,...,...,...,...,...
552,en,Metasequoia Forest Park,"481-72, Sangam-dong, Mapo-gu, Seoul","03900 95, Haneulgongwon-ro, Mapo-gu, Seoul",,,Open year round,,,"Subway Line 6, Mapo-gu Office Station, Exit 8"
553,ko,가을단풍길 - 남산남측순환로,서울 용산구 용산동2가 산 1-3,04340 서울 용산구 남산공원길 126 (용산동2가),02-2199-7623(용산구청 공원녹지과),https://www.seoul.go.kr/storyw/autumn/list.do,,,,
554,zh-CN,世界杯公园宠物游乐场,首尔市麻浦区上岩洞1535,03901 首尔市麻浦区缯山路32（上岩洞，询问中心）,+82-2-2124-2834,http://animal.seoul.go.kr/animalplay,10:00 ~ 21:00,周二至日,周一,6号线世界杯体育场站面号出口步行755米
555,ja,ワールドカップ公園ドッグラン,ソウル市マポ(麻浦)区サンアム(上岩)洞1535,03901 ソウル市マポ(麻浦)区チュンサンロ32 (サンアム(上岩)洞、案内所),+82-2-2124-2834,http://animal.seoul.go.kr/animalplay,10:00-21:00,火曜日~日曜日,月曜日,地下鉄6号線「ワールドカップキョンギジャン」駅1番出口から755m


# 3. PREPROCESSING
  1. 언어 칼럼의 데이터를 한글로 대치 (ex. ko -> 한국어)
  2. 언어별 구 추출

In [ ]:
seoul_nature_df = df

In [ ]:
# 언어에 따라 데이터 대치 (ex. ko -> 한국어)
for i in range(len(seoul_nature_df)):
  if seoul_nature_df['언어'][i] == 'ko': # 한국어
    seoul_nature_df['언어'][i] = '한국어'
  elif seoul_nature_df['언어'][i] == 'en': # 영어
    seoul_nature_df['언어'][i] = '영어'
  elif seoul_nature_df['언어'][i] == 'ja': # 일본어
    seoul_nature_df['언어'][i] = '일본어'
  else: # 중국어
    seoul_nature_df['언어'][i] = '중국어'

seoul_nature_df.head(5)

,언어,상호명,주소,신주소,전화번호,웹사이트,운영시간,운영요일,휴무일,교통정보
0,한국어,북한산국립공원,136-855 서울 성북구 정릉동 산 1-1,02700 서울 성북구 보국문로34길 6-21 (정릉동),02-909-0497,http://bukhan.knps.or.kr,04:00 ~ 17:00 (3월~11월)\r\n04:00 ~ 16:00 (12월~2월),매일,,"3호선 구파발역 1번 출구로 나와 704번, 34번 버스로 갈아타고 북한산성에서 하..."
1,일본어,ソニュド(仙遊島)公園,150-040 ソウル市永登浦区堂山洞1,07200 ソウル市ヨンドゥンポ(永登浦)区ソンユロ343 (堂山洞),+82-2-2634-7250,http://japanese.seoul.go.kr/%E7%94%9F%E6%B4%BB...,※公園 6:00-24:00／展示館 [夏期] 9:00-18:00、[冬期] 9:00-1...,,,地下鉄6号線「ハプジョン」駅8番出口からそれぞれ5714番バスに乗り、ソニュド(仙遊島)公園...
2,일본어,ソクチョン(石村)湖公園,138-915 ソウル市松坡区蚕室洞 47,05608 ソウル市ソンパグ(松坡区)サムハッサロ136 (蚕室洞),+82-2-412-0190,,,毎日,年中無休,2号線チャムシル駅2・3番出口
3,한국어,중랑천 제방,서울 중랑구 면목동 일대,서울특별시 중랑구 면목동 일대,02-938-9520~1,http://jr1000ecocenter.nowon.kr/main.do,,,,"7호선 중화 3번, 4번 출구 도보 10분\r\n\r\n1호선 신이문 3번 출구 도..."
4,한국어,구암공원,157-807 서울 강서구 가양동 1471,"07525 서울 강서구 허준로5길 42 (가양동, 구암공원관리사무실)",02-2600-4196,,,,,9호선 가양 1번


In [ ]:
# 주소에서 구 추출
seoul_nature_df['구'] = ''


for i in range(len(seoul_nature_df)):
  # '주소' 칼럼을 기준으로 하되, 비어있으면 '신주소' 칼럼을 이용해서 전체 주소 불러오기
  full_address = seoul_nature_df['주소'][i]
  if full_address == ' ':
    full_address = seoul_nature_df['신주소'][i]


  if seoul_nature_df['언어'][i] == '한국어': # 한국어 주소 추출
    try:
      seoul_nature_df['구'][i] = full_address.split(' ')[2]
      if seoul_nature_df['구'][i][-1] != '구':
        seoul_nature_df['구'][i] = 'error'
    except:
      seoul_nature_df['구'][i] = 'error'

  elif seoul_nature_df['언어'][i] == '영어': # 영어 주소 추출
    try:
      address = seoul_nature_df['주소'][i].split(',')
      seoul_nature_df['구'][i] = address[['-gu' in address[i] for i in range(len(address))].index(True)].replace(' ', '')
    except:
      seoul_nature_df['구'][i] = 'error'

  elif seoul_nature_df['언어'][i] == '일본어': # 일본어 주소 추출
    try:
      address1 = full_address.split('市')
      address2 = address1[1].split('区')
      seoul_nature_df['구'][i] = address2[0]+'区'
    except:
      seoul_nature_df['구'][i] = 'error'

  else: # 중국어 주소 추출
    if '市' in full_address:
      address1 = full_address.split('市')
      if '区' in full_address:
        address2 = address1[1].split('区')
        seoul_nature_df['구'][i] = address2[0] + '区'
      elif '區' in full_address:
        address2 = address1[1].split('區')
        seoul_nature_df['구'][i] = address2[0] + '区'
      else:
        seoul_nature_df['구'][i] = 'error'
      
    elif '首' in full_address:
      address1 = full_address.split('首')
      if '区' in full_address:
        address2 = address1[1].split('区')
        seoul_nature_df['구'][i] = address2[0][1:] + '区'
      elif '區' in full_address:
        address2 = address1[1].split('區')
        seoul_nature_df['구'][i] = address2[0][1:] + '区'
      else:
        seoul_nature_df['구'][i] = 'error'

In [ ]:
seoul_nature_df

,언어,상호명,주소,신주소,전화번호,웹사이트,운영시간,운영요일,휴무일,교통정보,구
0,한국어,북한산국립공원,136-855 서울 성북구 정릉동 산 1-1,02700 서울 성북구 보국문로34길 6-21 (정릉동),02-909-0497,http://bukhan.knps.or.kr,04:00 ~ 17:00 (3월~11월)\r\n04:00 ~ 16:00 (12월~2월),매일,,"3호선 구파발역 1번 출구로 나와 704번, 34번 버스로 갈아타고 북한산성에서 하...",성북구
1,일본어,ソニュド(仙遊島)公園,150-040 ソウル市永登浦区堂山洞1,07200 ソウル市ヨンドゥンポ(永登浦)区ソンユロ343 (堂山洞),+82-2-2634-7250,http://japanese.seoul.go.kr/%E7%94%9F%E6%B4%BB...,※公園 6:00-24:00／展示館 [夏期] 9:00-18:00、[冬期] 9:00-1...,,,地下鉄6号線「ハプジョン」駅8番出口からそれぞれ5714番バスに乗り、ソニュド(仙遊島)公園...,永登浦区
2,일본어,ソクチョン(石村)湖公園,138-915 ソウル市松坡区蚕室洞 47,05608 ソウル市ソンパグ(松坡区)サムハッサロ136 (蚕室洞),+82-2-412-0190,,,毎日,年中無休,2号線チャムシル駅2・3番出口,松坡区
3,한국어,중랑천 제방,서울 중랑구 면목동 일대,서울특별시 중랑구 면목동 일대,02-938-9520~1,http://jr1000ecocenter.nowon.kr/main.do,,,,"7호선 중화 3번, 4번 출구 도보 10분\r\n\r\n1호선 신이문 3번 출구 도...",중랑구
4,한국어,구암공원,157-807 서울 강서구 가양동 1471,"07525 서울 강서구 허준로5길 42 (가양동, 구암공원관리사무실)",02-2600-4196,,,,,9호선 가양 1번,강서구
...,...,...,...,...,...,...,...,...,...,...,...
552,영어,Metasequoia Forest Park,"481-72, Sangam-dong, Mapo-gu, Seoul","03900 95, Haneulgongwon-ro, Mapo-gu, Seoul",,,Open year round,,,"Subway Line 6, Mapo-gu Office Station, Exit 8",Mapo-gu
553,한국어,가을단풍길 - 남산남측순환로,서울 용산구 용산동2가 산 1-3,04340 서울 용산구 남산공원길 126 (용산동2가),02-2199-7623(용산구청 공원녹지과),https://www.seoul.go.kr/storyw/autumn/list.do,,,,,용산구
554,중국어,世界杯公园宠物游乐场,首尔市麻浦区上岩洞1535,03901 首尔市麻浦区缯山路32（上岩洞，询问中心）,+82-2-2124-2834,http://animal.seoul.go.kr/animalplay,10:00 ~ 21:00,周二至日,周一,6号线世界杯体育场站面号出口步行755米,麻浦区
555,일본어,ワールドカップ公園ドッグラン,ソウル市マポ(麻浦)区サンアム(上岩)洞1535,03901 ソウル市マポ(麻浦)区チュンサンロ32 (サンアム(上岩)洞、案内所),+82-2-2124-2834,http://animal.seoul.go.kr/animalplay,10:00-21:00,火曜日~日曜日,月曜日,地下鉄6号線「ワールドカップキョンギジャン」駅1番出口から755m,マポ(麻浦)区


In [ ]:
seoul_nature_df.to_excel('/content/drive/MyDrive/빅데이터통계분석/data/seoul_nature.xlsx')